# Gene deletions

In [1]:
# Packages
#import cobra
#from cobra.io import read_sbml_model, load_model
#from cobra import Model, Reaction, Metabolite

# Load the model
%run 00load_model.ipynb

In [2]:
# Calculate max biomass and undecyl prodigiosin (RED) production
model.solver = 'glpk'
with model:
    base_max_growth = model.slim_optimize()
    print("Max biomass growth:", base_max_growth)
    model.objective = model.reactions.DM_RED_c
    base_max_production = model.slim_optimize()
    print("Max RED production:", base_max_production)

Max biomass growth: 0.07461812924636277
Max RED production: 0.13190476190476172


Identification and knockout of genes associated with reactions that consume undecyl prodigiosin.

In [3]:
# Reactions with undecyl prodigiosin (RED)
RED_reactions = model.metabolites.get_by_id('red_c').reactions
RED_reactions

frozenset({<Reaction DM_RED_c at 0x20f5e157fd0>,
           <Reaction REDS37 at 0x20f5e10a0a0>,
           <Reaction REDt at 0x20f5e1572e0>,
           <Reaction SRUBSYN at 0x20f5e118070>})

In [4]:
# See the reactions
print("------------------")
print("RED reactions")
print("------------------")
for x in RED_reactions:
    print("%s : %s" % (x.id, x.reaction))

------------------
RED reactions
------------------
REDS37 : atp_c + h2o_c + mbc_c + up_c --> amp_c + 2.0 h_c + 2.0 pi_c + red_c
DM_RED_c : red_c --> 
REDt : red_c --> red_e
SRUBSYN : 2.0 h_c + o2_c + red_c --> 2.0 h2o_c + strprub_c


In [5]:
# Get the GPR code to use for the gene deletion
model.reactions.get_by_id("SRUBSYN")

Reaction identifier,SRUBSYN
Name,SRUBSYN
Memory address,0x020f5e118070
Stoichiometry,2.0 h_c + o2_c + red_c --> 2.0 h2o_c + strprub_c 2.0 H+ + O2 + undecylprodigiosin --> 2.0 H2O + Streptorubin B
GPR,SCO5897
Lower bound,0.0
Upper bound,1000.0


Once we perform the knockout, both the lower and the upper bound will be zero.

In [6]:
# Knockout  
print('---------------------')
print('Gene knockout targets')
print('---------------------')
print(cobra.manipulation.knock_out_model_genes(model, ["SCO5897"]))

---------------------
Gene knockout targets
---------------------


NameError: name 'cobra' is not defined

Because we get this AttributeError, we decided to perform the knockout manually:

In [7]:
model.reactions.get_by_id("SRUBSYN").upper_bound = 0.0
model.reactions.get_by_id("SRUBSYN")

Reaction identifier,SRUBSYN
Name,SRUBSYN
Memory address,0x020f5e118070
Stoichiometry,2.0 h_c + o2_c + red_c --> 2.0 h2o_c + strprub_c 2.0 H+ + O2 + undecylprodigiosin --> 2.0 H2O + Streptorubin B
GPR,SCO5897
Lower bound,0.0
Upper bound,0.0


The lower and upper bounds are now 0, thus confirming the correct knockout of the gene. The next step is to determine whether this change had an impact on the production of undecyl prodigiosin.

In [8]:
# Max biomass and RED production after knockout
model.solver = 'glpk'
with model:
    max_growth = model.slim_optimize()
    print("Max biomass growth:", max_growth)
    model.objective = model.reactions.DM_RED_c
    max_production = model.slim_optimize()
    print("Max RED production:", max_production)

Max biomass growth: 0.07461812924637035
Max RED production: 0.13190476190476194


In [10]:
# Change (%) max biomass growth and RED production
print("Change in max growth:", round((max_growth-(base_max_growth))/(base_max_growth) * 100, 2), "%")
print("Change in max production:", round((max_production-(base_max_production))/(base_max_production) * 100, 2), "%")

Change in max growth: 0.0 %
Change in max production: 0.0 %


There's no change in the production of undecyl prodigiosin when SCO5897 is knocked out.